In [31]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping

data = pd.read_csv('/content/drive/MyDrive/palm mind/titanic_train.csv')
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [32]:
# data cleaning
# filling empty age field value with average according to pclasses
def impute_age(cols):
    Age = cols[0]
    Pclass = cols[1]
    if pd.isnull(Age):
        if Pclass == 1:
            return 37
        elif Pclass == 2:
            return 29
        else:
            return 24
    else:
        return Age


data['Age'] = data[['Age', 'Pclass']].apply(impute_age, axis=1)

<ipython-input-32-3ff885b71ca9>:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  Age = cols[0]
<ipython-input-32-3ff885b71ca9>:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  Pclass = cols[1]


In [33]:
# getting dummy variables
#since sex is a categorical variable we need to convert it to numerical value
sex = pd.get_dummies(data['Sex'], drop_first=True)
embark = pd.get_dummies(data['Embarked'], drop_first=True)

In [34]:
# deleting unwanted field to remove model complexity
data.drop(['PassengerId','Name','Ticket','Cabin', 'Sex', 'Embarked',  ], axis=1, inplace=True)
data.head()

,Survived,Pclass,Age,SibSp,Parch,Fare
0,0,3,22.0,1,0,7.2500
1,1,1,38.0,1,0,71.2833
2,1,3,26.0,0,0,7.9250
3,1,1,35.0,1,0,53.1000
4,0,3,35.0,0,0,8.0500


In [35]:
y = data['Survived']
X = data.drop('Survived', axis=1)

In [36]:
# splitting data into train and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=101)

In [37]:
# Initializing the neural network model
model = Sequential()

# Input and first hidden layer
model.add(Dense(units=64, activation='relu', input_dim=X_train.shape[1]))

# Batch Normalization
model.add(BatchNormalization())

# Second hidden layer
model.add(Dense(units=32, activation='relu'))

# Dropout layer to prevent overfitting
model.add(Dropout(0.2))

# Output layer
model.add(Dense(units=1, activation='sigmoid'))

# Summary of the model
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_10 (Dense)                     │ (None, 64)                  │             384 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 64)                  │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_11 (Dense)                     │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_12 (Dense)                     │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,753 (10.75 KB)

 Trainable params: 2,625 (10.25 KB)

 Non-trainable params: 128 (512.00 B)

In [38]:
# Compiling model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [39]:
# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=32,
                    validation_data=(X_test, y_test), callbacks=[early_stopping])

Epoch 1/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 3s 24ms/step - accuracy: 0.5944 - loss: 0.7197 - val_accuracy: 0.6480 - val_loss: 0.6867
Epoch 2/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - accuracy: 0.6734 - loss: 0.6329 - val_accuracy: 0.6816 - val_loss: 0.6110
Epoch 3/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.6998 - loss: 0.5943 - val_accuracy: 0.6760 - val_loss: 0.6119
Epoch 4/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.6703 - loss: 0.6342 - val_accuracy: 0.6927 - val_loss: 0.6083
Epoch 5/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7136 - loss: 0.6091 - val_accuracy: 0.6927 - val_loss: 0.6117
Epoch 6/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7090 - loss: 0.5748 - val_accuracy: 0.6704 - val_loss: 0.6084
Epoch 7/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6993 - loss: 0.5854 - val_accuracy: 0.7095 - val_loss: 0.5972
Epoch 8/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7499 - loss: 0.5661 - val_accuracy: 0.7095 - val_l

In [40]:
# Predict on the test set
y_pred_nn = (model.predict(X_test) > 0.5).astype("int32")

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step


In [41]:
# Confusion Matrix
conf_matrix_nn = confusion_matrix(y_test, y_pred_nn)
print("Confusion Matrix:\n", conf_matrix_nn)

Confusion Matrix:
 [[89 10]
 [42 38]]


In [42]:
# Classification Report
class_report_nn = classification_report(y_test, y_pred_nn)
print("\nClassification Report:\n", class_report_nn)


Classification Report:
               precision    recall  f1-score   support

           0       0.68      0.90      0.77        99
           1       0.79      0.47      0.59        80

    accuracy                           0.71       179
   macro avg       0.74      0.69      0.68       179
weighted avg       0.73      0.71      0.69       179

